In [23]:
import pandas as pd
import importlib
import berechnen_wp, pv_profil, lastprofile_VDI4655, temperatur_aussen, try_region, heizkurve
importlib.reload(berechnen_wp)

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', 200, 3, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(40599)
df = lastprofile_VDI4655.get_lastprofile(w, s, twe, 200, TRY_region, 3)
df['T_aussen'] = temperatur_aussen.get_hourly_temperature(40599, 2014)
hz, T_soll, T_n_vor, T_n_rueck = heizkurve.get_heizkurve('Heizkörper', df['T_aussen'], T_n_aussen)
df['T_vor'] = hz['T_vor']
df['T_rueck'] = hz['T_rueck']
wp_groesse = berechnen_wp.get_waermepumpe(h)
heizleistung_auslegung = heizkurve.get_heizleistung(T_n_aussen, h, T_soll)
df = heizkurve.get_heizleistung_profil(df, heizleistung_auslegung)
df = heizkurve.get_cop(wp_groesse, df)
V_ps, PS_verlust, Q_ps = berechnen_wp.get_pufferspeicher(h, T_n_vor, T_n_rueck)
df, P_el, COP = berechnen_wp.ohne_pv(df, Q_ps, PS_verlust)
pv = pv_profil.get_pv_profil(40599, 2014, 10)
df_pv = berechnen_wp.mit_pv(df.copy(), pv)
df_pvbs = berechnen_wp.mit_pvbs(df.copy(), pv, 10, 11)

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [24]:
importlib.reload(berechnen_wp)
strompreis = 0.358
ergebnisse_pv = berechnen_wp.ersparnis_pv(df_pv, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse_pv)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 3676
Geladene PV-Strom in Wärmepumpe in kWh: 1714

Netzbezug in kWh: 6224
Einspeisung ins Netz in kWh: 7606

Stromkosten ohne PV in €/a: 4157.82
Stromkosten mit PV in €/a: 2228.19
Einspeisevergütung in €/a: 605.44
Stromkosten Einsparung in €/a: 2535.07


In [25]:
importlib.reload(berechnen_wp)
ergebnisse_bs = berechnen_wp.ersparnis_bs(df_pvbs, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse_bs)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 5861
Geladene PV-Strom in Wärmepumpe in kWh: 1714
Geladene PV-Strom in Batteriespeicher in kWh: 2098
Geladene BS-Strom in Wärmepumoe in kWh: 591

Netzbezug in kWh: 5920
Einspeisung ins Netz in kWh: 5421

Stromkosten ohne PV in €/a: 4157.82
Stromkosten mit PV & BS in €/a: 2119.36
Einspeisevergütung in €/a: 431.51
Stromkosten Einsparung in €/a: 2469.97


In [27]:
importlib.reload(berechnen_wp)
df_ev = berechnen_wp.mit_pvev(df.copy(), pv, True)

In [26]:
importlib.reload(berechnen_wp)
df_pvbsev = berechnen_wp.mit_pvbsev(df.copy(), pv, 10, 11, True)

In [28]:
importlib.reload(berechnen_wp)
ergebnisse_ev = berechnen_wp.ersparnis_ev(df_ev, 10, 0.358)
berechnen_wp.print_ersparnis(ergebnisse_ev)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
EV Strombedarf in kWh: 1994
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 3958
Geladene PV-Strom in Wärmepumpe in kWh: 1714
Geladene PV-Strom in Elektroauto in kWh: 282

Netzbezug in kWh: 9650
Einspeisung ins Netz in kWh: 7324

Stromkosten ohne PV in €/a: 4871.69
Stromkosten mit PV, BS & EV in €/a: 3454.7
Einspeisevergütung in €/a: 582.99
Stromkosten Einsparung in €/a: 1999.98


In [29]:
importlib.reload(berechnen_wp)
ergebnisse_evbs = berechnen_wp.ersparnis_evbs(df_pvbsev, 10, 0.358)
berechnen_wp.print_ersparnis(ergebnisse_evbs)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
EV Strombedarf in kWh: 1994
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 6423
Geladene PV-Strom in Wärmepumpe in kWh: 1714
Geladene PV-Strom in Batteriespeicher in kWh: 2469
Geladene PV-Strom in Elektroauto in kWh: 175
Geladene BS-Strom in Elektroauto in kWh: 837
Geladene BS-Strom in Wärmepumoe in kWh: 528

Netzbezug in kWh: 7383
Einspeisung ins Netz in kWh: 4859

Stromkosten ohne PV in €/a: 4871.69
Stromkosten mit PV, BS & EV in €/a: 2643.11
Einspeisevergütung in €/a: 386.78
Stromkosten Einsparung in €/a: 2615.36


In [ ]:
def mit_pvbsev(df, pv, anlage_groesse, battery_capacity, homeoffice):
    # Passe Index in PV and Index in df an
    pv.index = pv.index.tz_localize(None)
    df.index = pd.to_datetime(df.index)
    pv_aligned = pv.reindex(df.index).fillna(0)
    df['PV Ertrag'] = pv_aligned.values.astype(float)

    # EV Profil
    ev_profil = pd.read_csv(f'./Inputs/car_availability_homeoffice_{homeoffice}_2014.csv', index_col=0)
    ev_profil.index = pv.index.tz_localize(None)
    ev_profil.index = pd.to_datetime(df.index)
    ev_aligned = ev_profil.reindex(df.index).fillna(0)
    df['ev zuhause'] = ev_aligned['EV_at_home']
    df['ev distanz'] = ev_aligned['distance_travelled']
    
    # EV Spezifikationen
    ev_effizienz = 191 / 1000  # kWh per km
    batteriekapazitaet = 72  # kWh
    batterie_min = 10  # %
    batterie_max = 80  # %
    min_batterie_niveau = batterie_min / 100 * batteriekapazitaet  # kWh
    max_batterie_niveau = batterie_max / 100 * batteriekapazitaet # kWh
    max_ladeleistung = 11  # kW
    
    # Start EV SOC
    ev_soc = max_batterie_niveau

    # Neue Spalten für BS, EV und WP
    df['PV to WP'] = 0.0
    df['BS to WP'] = 0.0
    df['EV Ladung'] = 0.0
    df['EV SOC'] = 0.0 
    df['PV to EV'] = 0.0
    df['BS to EV'] = 0.0
    df['battery_charge'] = 0.0
    df['battery_discharge'] = 0.0
    df['battery_soc'] = 0.0
    df['überschuss'] = 0.0
    df['netzbezug'] = 0.0
    df['eigenverbrauch'] = 0.0
    
    # BS Spezifikationen
    c_rate = 1
    charge_efficiency = 0.96  # BYD HVS & HVM
    discharge_efficiency = 0.96
    min_soc = 1
    max_soc = battery_capacity
    battery_soc = 5  # Initial state of charge in kWh (50% of battery capacity)

    if anlage_groesse<battery_capacity:
        battery_capacity = anlage_groesse

    for i, row in df.iterrows():
        pv_ertrag = row['PV Ertrag']
        strombedarf = row['Strombedarf']
        ev_zuhause = row['ev zuhause']
        ev_distanz = row['ev distanz']
        p_el_wp = row['elekt. Leistungaufnahme']
        ladeleistung = 0
        pv_to_ev = 0
        pv_to_wp =  0
        netzbezug = 0
        ueberschuss = 0
        eigenverbrauch_str = 0
        eigenverbrauch_wp = 0
        eigenverbrauch_bs = 0
        eigenverbrauch_ev = 0
        charge_to_battery = 0
        discharge_from_battery = 0
        bs_to_wp = 0
        bs_to_ev = 0
        bs_to_str = 0

        # Step 1: Überschuss nach Strombedarf (Prio 1)
        if pv_ertrag >= strombedarf:
            ueberschuss = pv_ertrag - strombedarf
            eigenverbrauch_str = strombedarf
        else:
            ueberschuss = 0
            eigenverbrauch_str = pv_ertrag
            netzbezug = strombedarf - eigenverbrauch_str

        # Step 2: Überschuss für WP (Prio 2)
        if ueberschuss > 0 and p_el_wp > 0:
            if ueberschuss >= p_el_wp:
                pv_to_wp = p_el_wp
                ueberschuss -= p_el_wp
                eigenverbrauch_wp = pv_to_wp
            else:
                pv_to_wp = ueberschuss
                ueberschuss = 0
                eigenverbrauch_wp = pv_to_wp
                netzbezug += p_el_wp - pv_to_wp
        else:
            pv_to_wp = 0
            netzbezug += p_el_wp
        
        # Step 3: Überschuss 2 BS (Prio 3)
        if ueberschuss > 0:
            charge_potential = ueberschuss * charge_efficiency
            charge_to_battery = min(charge_potential, max_soc - battery_soc)
            battery_soc += charge_to_battery
            ueberschuss -= (charge_to_battery / charge_efficiency)
            eigenverbrauch_bs = (charge_to_battery / charge_efficiency)
        else:
            charge_to_battery = 0
        
        charging_power = c_rate * battery_soc * charge_efficiency  # kW
        
        # Step 4: Berechne EV Ladezustand
        if ev_zuhause == 0 and ev_distanz > 0:
            ev_verbrauch = ev_distanz*ev_effizienz
            if ev_soc - ev_verbrauch >= min_batterie_niveau:
                ev_soc -= ev_verbrauch
            else:
                ev_distanz = (ev_soc - min_batterie_niveau)/ev_effizienz
                ev_soc = min_batterie_niveau
        
        # Step 5: Lade EV wenn Zuhause (Prio 4)
        if ev_zuhause == 1 and ev_soc < max_batterie_niveau:
            ladeleistung = min(max_ladeleistung, max_batterie_niveau - ev_soc)
            if ueberschuss >= ladeleistung:
                # Überschuss reicht aus, um die gewünschte Ladeleistung abzudecken
                ev_soc += ladeleistung
                pv_to_ev = ladeleistung
                eigenverbrauch_ev = pv_to_ev
                ueberschuss -= ladeleistung
            elif ueberschuss > 0:
                # Teilweise Laden mit Überschuss, Rest aus dem Netz
                pv_to_ev = ueberschuss
                netzbezug += ladeleistung - ueberschuss
                ev_soc += ladeleistung
                eigenverbrauch_ev = pv_to_ev
                ueberschuss = 0
            else:
            # Kein Überschuss, Laden komplett aus dem Netz
                ev_soc += ladeleistung
                netzbezug += ladeleistung
        
        # Step 6: BS entladen oder von Netz beziehen
        if netzbezug > 0:
            # Entlade Batterie, um Energiemangel zu decken
            discharge_needed = min(netzbezug / discharge_efficiency, charging_power)
            discharge_from_battery = min(discharge_needed, battery_soc - min_soc)
            energy_from_battery = discharge_from_battery * discharge_efficiency
            battery_soc -= discharge_from_battery
            # Übrige Energiemangel nach Batterienetladung = Netzbezug
            remaining_shortfall = netzbezug - energy_from_battery
            netzbezug = max(remaining_shortfall, 0) 

            # Zuweisung von BS Entladung basierend auf Priorität
            netzbezug_wp = max(0, p_el_wp - pv_to_wp)
            netzbezug_ev = max(0, ladeleistung - pv_to_ev)
            netzbezug_str = max(0, strombedarf-pv_ertrag)
            # Verfügbarer Strom aus der Batterie
            remaining_battery = energy_from_battery
            # 1. Prio: Netzbezug für Strombedarf
            if remaining_battery >= netzbezug_str:
                bs_to_str = netzbezug_str
                remaining_battery -= netzbezug_str
            else:
                bs_to_str = remaining_battery
                remaining_battery = 0
            # 2. Prio: Netzbezug für WP
            if remaining_battery > 0:
                if remaining_battery >= netzbezug_wp:
                    bs_to_wp = netzbezug_wp
                    remaining_battery -= netzbezug_wp
                else:
                    bs_to_wp = remaining_battery
                    remaining_battery = 0
            # 3. Prio: Netzbezug für EV
            if remaining_battery > 0:
                if remaining_battery >= netzbezug_ev:
                    bs_to_ev = netzbezug_ev
                    remaining_battery -= netzbezug_ev
                else:
                    bs_to_ev = remaining_battery
                    remaining_battery = 0
        else:
            discharge_from_battery = 0
            netzbezug = 0
        
        ueberschuss = max(0, ueberschuss)
        eigenverbrauch = eigenverbrauch_str+eigenverbrauch_wp+eigenverbrauch_bs+eigenverbrauch_ev
        # Step 7: Überprüfe Überschuss und Netzbezug
        if ueberschuss > 0 and netzbezug > 0:
            if netzbezug >= ueberschuss:
                netzbezug -= ueberschuss
                eigenverbrauch += ueberschuss
                ueberschuss = 0
            else:
                ueberschuss -= netzbezug
                eigenverbrauch += netzbezug
                netzbezug = 0

        df.at[i, 'ev distanz'] = ev_distanz
        df.at[i, 'überschuss'] = float(ueberschuss)
        df.at[i, 'PV to WP'] = float(pv_to_wp)
        df.at[i, 'PV to EV'] = pv_to_ev
        df.at[i, 'EV SOC'] = ev_soc
        df.loc[i, 'EV Ladung'] = ladeleistung
        df.at[i, 'eigenverbrauch'] = float(eigenverbrauch)
        df.at[i, 'netzbezug'] = netzbezug
        df.loc[i, 'battery_charge'] = charge_to_battery
        df.loc[i, 'battery_discharge'] = discharge_from_battery
        df.loc[i, 'BS to WP'] = bs_to_wp
        df.loc[i, 'BS to EV'] = bs_to_ev
        df.loc[i, 'BS to Haushalt'] = bs_to_str
        df.loc[i, 'battery_soc'] = battery_soc
    return df